# funcX Tutorial

funcX is a Function-as-a-Service (FaaS) platform for science that enables you to convert almost any computing resource into a high-performance function serving device. Deploying a funcX endpoint will integrate your resource into the function serving fabric, allowing you to dynamically send, monitor, and receive results from function invocations. funcX is built on top of Parsl, allowing you to connect your endpoint to large compute resources via traditional batch queues, where funcX will dynamically provision, use, and release resources on-demand to fulfill function requests.

Here we provide an example of using funcX to register a function and run it on a publicly available tutorial endpoint.

We start by creating a funcX client to interact with the service.

In [1]:
from funcx.sdk.client import FuncXClient

fxc = FuncXClient()

### Registering a function

Here we define an example function to be used. This function will report the name of the machine it is executed on. funcX functions can be applied to almost any endpoint the user is permitted to use, so we can reuse this function across endpoints.

In [3]:
def platform(items):
    import platform
    return platform.uname()

We first register the function <b>WITHOUT</b> a container. This means the function will be executed with either the default python environment the worker is deployed in, or, in the case of a Kubernetes endpoint, the default container used by the endpoint.

In [4]:
func_uuid1 = fxc.register_function(platform,
                                   description="A function to list platform info")
print(func_uuid1)

62966e2c-7b84-416e-ad37-ac1341a14cae


Now that the function has been registered with the funcX service we can invoke it.

This code specifies the public funcX tutorial endpoint. Any user is permitted to use this endpoint. The function returns a task_id that can subsequently be used to retrieve the result of its execution.

In [5]:
payload = []

endpoint_uuid = '4b116d3c-1703-4f8f-9f6f-39921e5864df' # Public tutorial endpoint

res1 = fxc.run(payload, endpoint_id=endpoint_uuid, function_id=func_uuid1)
print(res1)

a5389269-a992-4563-8800-a6141378d1ee


In [6]:
fxc.get_result(res1)

uname_result(system='Linux', node='ip-10-0-1-192.ec2.internal', release='4.14.152-127.182.amzn2.x86_64', version='#1 SMP Thu Nov 14 17:32:43 UTC 2019', machine='x86_64', processor='')

Note that the function's result shows that the function was executed on an Amazon EC2 machine. 

### Running on River

Now we can try running the function on the River K8s cluster. To do this we simply change the endpoint_uuid and the function will be performed using the default container.

### IMPORTANT: The River endpoint is restricted to members of a Globus Auth Group.  To join this group please email Zhuozhao (zhuozhao@uchicago.edu).

In [7]:
river_ep_uuid = '2c92a06a-015d-4bfa-924c-b3d0c36bdad7' # River endpoint

res2 = fxc.run(payload, endpoint_id=river_ep_uuid, function_id=func_uuid1)
print(res2)

22fc8c75-b624-46e5-a78c-339a6c6136b0


In [8]:
fxc.get_result(res2)

uname_result(system='Linux', node='funcx-river-1581556517816', release='5.3.2-1.el7.elrepo.x86_64', version='#1 SMP Tue Oct 1 08:18:21 EDT 2019', machine='x86_64', processor='')

Note that the result now shows the function was executed on River.

## Registering a function WITH a contontainer.

When registering a function you can associate a specific container with it. If a container is specified, the container will be staged to the endpoint and used to run the function. Here we show how a container is included in the function registration.

In [9]:
func_uuid2 = fxc.register_function(platform,
                                   container_uuid='00d68bd4-ca6a-4804-a6fa-6ed23b23f607',   # A docker container
                                   description="A function to list platform info")
print(func_uuid2)

6f72610f-238d-4223-8953-4b7a9d3f7d20


In [11]:
payload = [1]

endpoint_uuid = "2c92a06a-015d-4bfa-924c-b3d0c36bdad7" # River endpoint

res3 = fxc.run(payload, endpoint_id=river_ep_uuid, function_id=func_uuid2)
print(res3)

e7a9772f-0f10-4117-826a-bb43d7bf9aad


It may take sometime to get the result at the first time, since the endpoint needs to start a pod to run the function for the 

In [12]:
fxc.get_result(res3)

uname_result(system='Linux', node='funcx-river-1581556699844', release='5.3.2-1.el7.elrepo.x86_64', version='#1 SMP Tue Oct 1 08:18:21 EDT 2019', machine='x86_64', processor='')